In [1]:
import billboard
import pandas
import lyricwikia
import time
import pymongo

In [2]:
def doworkson(hotchart, chartname):
    
    #takes the instance data and gets what he needs.
    #requires fresh instance from billboard.py hotchart
    #and a user chartname    
        
    #holder for unique artists in each chart....we can expand upon this later if we want more derived stats
    artists = []
    
    #gets current chart
    currentChartWeek = hotchart.date

    #creates dictionary based on unique inputs
    keys = [currentChartWeek, chartname]
    values = [currentChartWeek, []]
    
    #creation of a very hot dictionary
    hottydict = dict(zip(keys, values))

    #squeezes the data juices
    for hotty in hotchart:

        title = hotty.title
        artist = hotty.artist
        
        #we can talk to expand derived stats to get more data juices
        if artist not in artists:

            artists.append(artist)
        
        #songweeks = weeks on chart
        songweeks = hotty.weeks
        
        #peakPos = peak spot on chart
        peakPos = hotty.peakPos
        
        #lastPos = last weeks spot on chart
        lastPos = hotty.lastPos
        
        #rank = rank on this chart
        rank = hotty.rank
        
        #isNew = isNew true or false
        isNew = hotty.isNew

        #caliente=hot
        hottydict[chartname].append(dict(
                                      Song_title = title,
                                      Song_artist = artist,
                                      Song_duration_weeks = songweeks,
                                      Song_peak = peakPos,
                                      Song_last_week = lastPos,
                                      Song_rank = rank,
                                      Song_isNew = isNew,
                                        )
                                    )
    #throw on the unique artist list
    hottydict['featured_artists_billboard_hot100'] = artists
    
    #creates and appends derived stat
    artistmixes = len(artists)
    top100 = len(hotchart)
    total_artists_featured_perc = artistmixes/top100
    hottydict['total_artists_featured_perc'] = total_artists_featured_perc
    
    return hottydict

In [3]:
def mongoify(done_son, collectionname):
    return db[collectionname].insert_one(done_son)

In [4]:
def doing_work(chartname, collectionname):
    ###this guy runs the show, take him and feed him the chartname from billboard.com you want to scrape
    ###he also needs a collectionname to be happy so pick a name to remember for mongo. We can reference to pull from later    
    
    
    hotchart = billboard.ChartData(chartname)
    ###the beauty is this guy sucks em dry. They're are no time filters but we'll get them through mongo later
    while hotchart.previousDate:
        
        #done_son = hottydict -result of function doworkson and contains the meat
        done_son = doworkson(hotchart, chartname)
        mongoify(done_son, collectionname)


        hotchart = billboard.ChartData(chartname, hotchart.previousDate)
        
    return

In [5]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.christmascarol

In [6]:
#####CHANGE CHARTNAME TO WHAT YOU WANT
#####CHANGE COLLECTIONNAME TO WHAT YOU WANT

###select chart to scrape - chartname
# chartname = 'billboard-200' ---scraped
chartname = "pop-songs"

# ### change collection name
# collectionname = 'billy_boards_200' ---scraped
collectionname = "pop_songs"

doing_work(chartname, collectionname)





KeyboardInterrupt: 